<div class="alert alert-block alert-success"> 
<h1>Задания 5.2</h1>
</div>

Нужно выполнить задачу от руководительницы отдела контент-маркетинга новой онлайн-школы английского языка. Она хочет составить ТЗ копирайтеру на написание постов, но для этого ей нужно знать, какие типы постов популярны. За завтраком вы с ней обсуждали твое последнее занятие с курса Python, где мы парсили страницы сообществ ВК, и у вас появилась идея – проанализировать самые успешные посты известной онлайн-школы Skyeng  в [сообществе Skyeng в ВК](https://vk.com/skyeng).

Теперь перед тобой стоит задача – выгрузить маркетологам для анализа 50 самых популярных постов за все время существования сообщества. Метрику популярности будем считать как сумму лайков, репостов и комментариев к каждому посту. Если останется время, для дополнительного анализа выгрузи все комментарии к этим постам в отдельный файл.

---

## Задание 1

Получи собственный `access_token` в ВК. Важно пройти этот путь самостоятельно, можешь использовать документацию API ВК или рекомендации из ноутбука занятия.  Токен присылай вместе с файлом ноутбука в файле с именем `my_own_token.py`. Обещаем не делать ничего противозаконного.

*Если тебе некомфортно, можешь не присылать токен нам. Однако получить его все же обязательно – это необходимо для полноценного освоения темы урока, а также для выполнения заданий ниже.*

In [1]:
# читаем токен из файла my_own_token.py, куда мы сохранили access_token
from my_own_token import access_token as token

---

## Задание 2

__Сформируй запрос, который получает 100 записей со стены сообщества__

1. Найди в документации метод, который отдает все записи со стены сообщества Skyeng: https://vk.com/skyeng
2. Создай переменую для URL 
3. Создай переменную с параметрами
4. Создай переменную запроса
5. Создай переменную, в которой будут данные ответа в формате словаря

In [2]:
import requests 
import pandas as pd

Справка:
1. [создание токена](https://vk.com/apps?act=manage)
1. [методы API](https://vk.com/dev/methods)
1. [ограничения и рекомендации](https://vk.com/dev/api_requests?f=3.%20%D0%9E%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%87%D0%B5%D0%BD%D0%B8%D1%8F%20%D0%B8%20%D1%80%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8)

In [3]:
# нахожу id сообщества по имени

url = 'https://api.vk.com/method/groups.getById'

params = {
    'access_token': token,
    'v': '5.103',
    'group_id': 'skyeng',
    'fields': 'id'
}
req = requests.get(url, params)
res = req.json()
id_skyeng = res['response'][0]['id']

In [4]:
# нахожу количество записей на стене сообщества (ограничение по документации 100)

url = 'https://api.vk.com/method/wall.get'

params = {
    'access_token': token,
    'v': '5.103',
    'owner_id': -id_skyeng,
    'count': 100 # согласно документации для сообществ необходимо указывать со знаком «-», integer
}
req = requests.get(url, params)
res = req.json()

print('Кол-во записей:', len(pd.DataFrame(res['response'])))

Кол-во записей: 100


---

## Задание 3

Выгрузи в переменную `posts` в формате датафрейма __все записи__ сообщества Skyeng: https://vk.com/skyeng, 

1. Используй цикл `While`
2. Для выгрузки __всех записей__ тебе поможет параметр `offset`. 
3. Записей больше 7000! Не забудь про ограничение "не больше 3 запросов в секунду" от API VK, здесь тебе поможет библиотека `time`

In [5]:
import time # чтобы спать

In [6]:
# выгружаю в переменную posts в формате датафрейма все записи сообщества https://vk.com/skyeng

offset = 0
max_count = 100
posts  = pd.DataFrame()

while True:      # повторяю цикл пока он не остановится по условию ниже
    
    url = 'https://api.vk.com/method/wall.get'
    params = {
        'access_token': token,
        'v': '5.103',
        'owner_id': -id_skyeng,
        'count': max_count,
        'offset': offset
    }
    
    req = requests.get(url=url, 
                       params=params)
    res = req.json()
    
    if len(res['response']['items']) != 0:      # проверяю вернулись ли нам хоть какие-нибудь записи
        posts = pd.concat([posts, pd.DataFrame(res['response']['items'])]) 
    else:
        break                                   # если ничего не вернулось, то прервать цикл
#     print('Уже скачано', offset, 'постов')  # не буду показывать вывод, чтобы не засорять блокнот
    offset = offset + max_count                 # Смещаем offset на число возвращенных записей
    time.sleep(0.5)                             # Опять контролируем частоту запросов, чтобы нас не ограничили

print('Кол-во постов:', len(posts))

Кол-во постов: 7956


In [7]:
req.connection.close() # закрываю соединение

In [8]:
posts.sample(3)

,id,from_id,owner_id,date,marked_as_ads,post_type,text,is_pinned,attachments,post_source,comments,likes,reposts,views,is_favorite,edited,hash,copy_history,signer_id,geo
21,5728,-51890028,-51890028,1455566437,0,post,,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},"{'can_post': 1, 'count': 0, 'groups_can_post':...","{'can_like': 1, 'count': 18, 'user_likes': 0, ...","{'count': 10, 'user_reposted': 0}",NaN,False,NaN,99PsosUkHkKTeoU_pRU8mIYLnkU,NaN,NaN,NaN
29,6147,-51890028,-51890028,1458295821,0,post,Умение вести беседу на английском во время сов...,NaN,"[{'type': 'link', 'link': {'url': 'http://skye...",{'type': 'vk'},"{'can_post': 1, 'count': 0, 'groups_can_post':...","{'can_like': 1, 'count': 21, 'user_likes': 0, ...","{'count': 22, 'user_reposted': 0}",NaN,False,NaN,hx6zmjaJ1w3DUkMBqx6gnXKMHUU,NaN,NaN,NaN
43,10138,-51890028,-51890028,1503337140,1,post,[club4325149|The Village] рассказали о первом ...,NaN,"[{'type': 'link', 'link': {'url': 'http://www....",{'type': 'vk'},"{'can_post': 1, 'count': 0, 'groups_can_post':...","{'can_like': 1, 'count': 11, 'user_likes': 0, ...","{'count': 0, 'user_reposted': 0}",{'count': 2174},False,NaN,KwE-2C6moZ0dtT_G2dyzLZNyflc,NaN,NaN,NaN


---

## Задание 4

Выгрузи 50 самых популярных постов. Метрикой популярности мы считаем сумму лайков, репостов и комментариев.

1. Создай столбцы с числовыми значениями для лайков, репостов и комментариев и забери из словаря только значение по ключу `count` 
2. Создай отдельный столбец, где будет сумма пользовательских активностей для каждого поста
3. Отсортируй дата-фрейм по столбцу с суммой пользовательских активностей
4. Выбери 50 самых популярных постов

Если задание кажется легким, реализуй тот же код с `lambda` функцией.

In [9]:
# вытаскиваю из словарей столбцов comments, likes, reposts только нужные значения

# заполняю даными с прошлого шага новые столбцы
posts['comments_count'] = posts['comments'].apply(lambda x: x['count'])
posts['likes_count'] = posts['likes'].apply(lambda x: x['count'])
posts['reposts_count'] = posts['reposts'].apply(lambda x: x['count'])

In [10]:
# заполняю данными столбец с суммой по всем активностям

posts['activity'] = posts['comments_count'] + posts['likes_count'] + posts['reposts_count']

In [11]:
# вывожу Tоп-50 по активности постов

posts.sort_values(by=['activity'], ascending=False).head(50)

,id,from_id,owner_id,date,marked_as_ads,post_type,text,is_pinned,attachments,post_source,...,is_favorite,edited,hash,copy_history,signer_id,geo,comments_count,likes_count,reposts_count,activity
55,63081,-51890028,-51890028,1602590409,0,post,КОНКУРС! Выиграйте уроки английского и бьюти-б...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,NaN,bk6VxZOqwPEyqWSA3OJXvs17-NM,NaN,NaN,NaN,10962,117,2,11081
2,25259,-51890028,-51890028,1554800244,0,post,Через 15 дней вы накачаете пресс и выучите анг...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,1.554800e+09,txXSN603EOeFMCNso1hjS7__d5M,NaN,NaN,NaN,239,6997,1444,8680
49,977,-51890028,-51890028,1416930109,0,post,100 полезных английских фраз\nЭти фразы выруча...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,1.416991e+09,a_lLuWO0j5XrDP2yLk9ZZMBqJ8Y,NaN,NaN,NaN,0,4012,975,4987
90,75933,-51890028,-51890028,1604923204,0,post,КОНКУРС! Выиграйте уроки английского и боксы с...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,NaN,OwQ5EwveK13_0r4ior81LKr2IFY,NaN,NaN,NaN,2891,133,6,3030
81,42960,-51890028,-51890028,1568444249,0,post,😑Предлоги в английском — больная тема для всех...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,1.568449e+09,-FC0cY8Vzc9Gn6s_c-wZW5swhBM,NaN,NaN,NaN,50,1918,437,2405
46,48883,-51890028,-51890028,1574414757,0,post,"Кот Виктор показывает, как стал причиной сканд...",NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,NaN,9FXOxZ0iAtyzsYzIyvbDrm_yk_Q,NaN,NaN,NaN,54,1620,210,1884
53,31067,-51890028,-51890028,1561301735,0,post,Играем в слова. У кого словарный запас больше?...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,1.561308e+09,LabK_uJr2e-Q6SdeYvJ9sOljgt8,NaN,NaN,NaN,1634,135,3,1772
86,55740,-51890028,-51890028,1587207600,0,post,Без фразовых глаголов можно понимать только са...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,NaN,gSWT0kyK3FKiSzVlcCk7TOc1FRk,NaN,NaN,NaN,36,1397,275,1708
14,51793,-51890028,-51890028,1581057787,0,post,"Английский снова сводит нас с ума, 😉и ему вооб...",NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,1.581063e+09,k7vVWuZzyHvlbvBC0afh2oaOyMM,NaN,NaN,NaN,111,1383,198,1692
40,57606,-51890028,-51890028,1589373062,0,post,Готовы ли вы к жизни в англоговорящем городе? ...,NaN,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'type': 'vk'},...,False,NaN,uWtVPbQW0TXYUIEiyqmDcZszlLs,NaN,NaN,NaN,84,1479,124,1687


---

## Задание 5

__Почисти данные, сохрани результат в файл для отправки копирайтерам__

1. Отфильтруй в датафрейме только нужные столбцы:
    - дата поста
    - текст поста
    - количество комментариев
    - количество лайков
    - количество репостов
    - общее количество пользовательских активностей

2. Выгрузи датафрейм в .csv файл


In [12]:
# фильтрую столбцы

cleared_post_data  = posts[['date', 'text', 'comments_count','likes_count', 'reposts_count', 'activity']].copy()

In [13]:
# перевожу дату из формата unix в datetime

cleared_post_data['date'] = pd.to_datetime(cleared_post_data['date'], unit='s')

In [14]:
cleared_post_data

,date,text,comments_count,likes_count,reposts_count,activity
0,2022-02-18 15:48:35,"Никаких бритв, носков и гелей для душа 🙅🏼‍♀\nЛ...",0,5,3,8
1,2022-03-01 07:23:52,В русском языке правилам пунктуации уделяют мн...,5,39,25,69
2,2022-02-27 19:05:56,Даже профессиональные учителя иногда совершают...,0,55,13,68
3,2022-02-26 15:00:23,"Английские прилагательные, чтобы описать эмоци...",2,215,261,478
4,2022-02-25 17:29:30,"Бывают ситуации, которые вы не можете поменять...",0,62,34,96
...,...,...,...,...,...,...
51,2013-12-03 09:13:49,Приходите на онлайн встречу (goo.gl/4eKCYD) ра...,0,2,0,2
52,2013-08-26 06:34:18,Мы начинаем серию интервью с преподавателями ш...,0,3,0,3
53,2013-05-28 18:40:29,В прошедшее воскресенье у нас прошла первая вс...,4,4,0,8
54,2013-04-22 12:41:53,Изучение английского по скайпу в школе Skyeng,0,7,0,7


In [15]:
# сохраняю результаты в файл Excel
cleared_post_data.to_excel('cleared_post_data.xlsx')

---

## EXTRA

__К 50 самым популярным постам выгрузи все комментарии, чтобы оценить, как пользователи восприняли посты__

1. Создай цикл, который проходит по всем `post ID`
2. В каждом цикле собери все комментарии к посту, используя `offset`
3. В каждом цикле достань лайки в отдельный столбец
4. Собери все комментарии в одном датафрейме



*Обрати внимание, что для работы с постами нужны __дополнительные параметры в токене__*

In [16]:
import json

In [17]:
# выгружаю в переменную top50_posts Tоп-50 по активности постов

top50_posts = posts.sort_values(by=['activity'], ascending=False).head(50)

In [21]:
# выгружаю все комментарии к постам Top-50

comments_posts  = pd.DataFrame()

for post_id in top50_posts['id']: # перебираю Top-50 постов в сообществе
    
    offset = 0 # начальное смещение
    max_count = 100 # число комментариев, которое возможно получить за 1 раз
    comments_field_text =  [] # в этот список буду добавлять текст комментариев для каждого поста
    number_of_likes_per_comment = 0 # число лайков за каждый комментарий
    
    while True:
        url = 'https://api.vk.com/method/wall.getComments'
        params = {
            'access_token': token,
            'v': '5.103',
            'owner_id': -id_skyeng,
            'post_id': post_id,
            'count': max_count,  # по умолчанию 10, max - 100
            'offset': offset,
            'need_likes': 1 # мне необходима дополнительная информация по лайкам комментариев
        }
        
        
        # словарь с непосредственными данными в постах
        response = requests.get(url, params=params).json()['response']['items']
        
        
        if len(response) != 0:      # проверяю вернулись ли хоть какие-нибудь записи
            try: # пробую выполнить код
                
                field_text = response[0]['text'] # поле текстового комментария для поста
                comments_field_text.append(field_text) # добавляю значение комментария в список
                
                field_like = response[0]['likes']['can_like'] # поле для лайков в комментарии
                number_of_likes_per_comment += field_like # cумма лайков для каждого комментария
                
            except: # если вылетает ошибка — присваиваю значение ноль
                comments_field_text.append('0')
        else:
            break # если ничего не вернулось, то прерываю цикл
    
        offset = offset + max_count    # смещаю offset на число возвращенных записей
        time.sleep(0.5)                # контролирую частоту запросов, чтобы не забанили  
        
#         print('post_id = ', post_id, 'offset = ', offset) # показываю, что цикл не завис
        
    # формирую словарь названий столбцов в итоговой таблице
    d = {'post_id': post_id, 'text': [comments_field_text], 'likes': number_of_likes_per_comment}
    
    # дописываю новые строки для каждого поста в итоговую таблицу
    comments_posts = pd.concat([comments_posts, pd.DataFrame(data = d)])  
        
print('Кол-во постов:', len(comments_posts))

Кол-во постов: 50


In [22]:
pd.set_option('max_colwidth', None) # хочу видеть всё содержимое ячеек в датафрейме
comments_posts.set_index('post_id')

,text,likes
post_id,,
63081,"[+, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, ➕, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, ➕, +, +, +, +, +, +, +, ➕, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, +, ➕, +, +, +, +, +, +, +, +, +, +, +, +, +, ➕, +, +, +, +, ➕, ➕, ...]",110
25259,"[Ееееееее, спасибо! Я счастлив, как слон!!!, 👍]",2
977,[],0
75933,"[➕, +, +, +, ➕, ➕, ➕, +, +, +, +, +, ➕, +, +, +, +, +, ➕, +, ➕, ➕, ➕, +++, ➕, +, +, +, +]",29
42960,"[, 0]",0
48883,[😄],1
31067,"[Goodbay, Remove удалить, Rat, Giggle хохотать, А дерево уже было), Likely правдоподобно, Tense напряжённый, Trouble, Year (год), Yell ругаться]",10
55740,[🔥],1
51793,[+],1
